In [37]:
import numpy as np
from collections.abc import Iterable
import numbers
import jax.numpy as jnp
from jax import grad

In [59]:
class MLP_with_backpropagation():
    def identity(x):
        return x
        
    def __init__(self, shape, activations = None): #len(activations)+1=len(shape)
        fed_values = []
        activation_values = []
        for layer_size in shape:
            fed_values.append(np.array([0]*layer_size))
            activation_values.append(np.array([0]*layer_size))
        self.fed_values = fed_values
        self.activation_values = activation_values
        #first layer values are set so that indexes match
        weights = [0]
        biases = [0]
        for i in range(1, len(self.fed_values)):
            n = len(self.fed_values[i])
            m = len(self.fed_values[i-1])
            #initialising with random values
            weight_matrix = np.random.normal(0,1,(n,m))
            weights.append(weight_matrix)
            bias_vector = np.random.normal(0,1,n)
            biases.append(bias_vector)
        self.weights = weights
        self.biases = biases
        
        if activations:
            self.activations = [0] + [np.vectorize(activation) for activation in activations]
            self.activations_sv = [0] + [activation for activation in activations]
        else:
            self.activations = [0] + [np.vectorize(MLP_with_backpropagation.identity)] * (len(shape) - 1)
            self.activations_sv = [0] + [MLP_with_backpropagation.identity] * (len(shape) - 1)
            
    #2 functions below are only for technical purposes
    def is_iterable(obj):
        return isinstance(obj, Iterable)

    def is_numeric_vector_of_given_length(supposed_vector, length):
         if not MLP_with_backpropagation.is_iterable(supposed_vector):
             return False
         if len(supposed_vector) != length:
             return False
         for el in supposed_vector:
             if not isinstance(el, numbers.Number):
                 return False
         return True
        
    def set_input(self, inputt):
        if not MLP_with_backpropagation.is_numeric_vector_of_given_length(inputt, len(self.fed_values[0])):
            print("Wrong input size or type, it is supposed to be a numerical list or a 1D np.array of length of 1st layer")
            return False
        self.fed_values[0] = np.array(inputt)
        self.activation_values[0] = np.array(inputt) #input is input
        return True
        
    def feed_forward(self):
        for i in range(1, len(self.fed_values)):
            self.fed_values[i] = np.dot(self.weights[i], self.activation_values[i-1]) + self.biases[i] ##can do if statement if activation is function of layer not neuron
            self.activation_values[i] = self.activations[i](self.fed_values[i])

    def predict(self, x):
        if not self.set_input(x):
            return False
        self.feed_forward()
        return self.activation_values[-1]
        
    #manual setting of weights and biases    
    def set_weights(self, weights):
        self.weights = weights

    def set_biases(self, biases):
        self.biases = biases

    def squared_error(pred, expected):
        return (pred - expected)**2

    
    #returns a pair first element is for weights second for biases
    def derivative(self, inputt, expected):
        self.predict(inputt)
        dx = 10**(-6)
        weight_grad = [0] + [np.zeros(self.weights[i].shape) for i in range(1, len(self.weights))]
        bias_grad = [0] + [np.zeros(len(self.biases[i])) for i in range(1, len(self.biases))]
        neuron_activation_grad = [0] + [np.zeros(len(self.activation_values[i])) for i in range(1, len(self.activation_values))]
        neuron_fed_grad = [0] + [np.zeros(len(self.fed_values[i])) for i in range(1, len(self.fed_values))]
        last_layer = True
        for i in range(len(self.fed_values)-1, 0, -1):
            #derivatives in respect to neuron activation values
            for j in range(len(self.activation_values[i])):
                if last_layer:
                    x_0 = self.activation_values[i][j]
                    y_0 = expected[j]
                    dc_da = grad(MLP_with_backpropagation.squared_error, argnums = 0)
                    neuron_activation_grad[i][j] = dc_da(x_0, y_0)
                else:
                    neuron_activation_grad[i][j] = sum([neuron_fed_grad[i+1][k] * self.weights[i+1][k][j] for k in range(len(neuron_fed_grad[i+1]))])
                
                da_dz = grad(self.activations_sv[i], argnums = 0)                                       
                neuron_fed_grad[i][j] = neuron_activation_grad[i][j]*da_dz(self.fed_values[i][j])
                bias_grad[i][j] = neuron_fed_grad[i][j]
                
                for k in range(len(self.weights[i][j])):
                    weight_grad[i][j][k] = neuron_fed_grad[i][j] * self.activation_values[i-1][k]
            last_layer = False
        return (weight_grad, bias_grad)
                
        

    
            
        

In [26]:
for i in range(10,0, -1):
    print(i)


    

10
9
8
7
6
5
4
3
2
1


In [61]:
network = MLP_with_backpropagation([3,4,5])
w_grad, b_grad = network.derivative(np.array([1,2,3]), np.array([3,4,5,6,7]))



In [63]:
print(b_grad)

[0, array([ 29.57564735,  43.02214813, -20.65423393,  17.97513199]), array([-15.18190479,   0.3594265 , -27.13799858, -12.19403458,
         0.27427387])]


In [32]:
def difference_squared(x, y):
    return (x-y)**2

In [33]:
print(difference_squared(7,3))

16


In [44]:
df_dx = grad(difference_squared, argnums = 0)
val = df_dx(7.0,3.0)
print(df_dx(7.0,3.0)*7)

56.0
